In [39]:
%load_ext autoreload

%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [43]:
import torch
from glasses.nn.models.classification.inception import *

In [44]:
x = torch.rand(1, 32, 48, 48)

In [45]:
InceptionABlock(32)

InceptionABlock(
  (block): InputForward(
    (blocks): ModuleList(
      (0): Sequential(
        (conv1): Conv2dPad(32, 64, kernel_size=(1, 1), stride=(1, 1))
        (conv2): Conv2dPad(64, 96, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (conv3): Conv2dPad(96, 96, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      )
      (1): Sequential(
        (conv1): Conv2dPad(32, 64, kernel_size=(1, 1), stride=(1, 1))
        (conv2): Conv2dPad(64, 96, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      )
      (2): Sequential(
        (pool): AvgPool2d(kernel_size=3, stride=1, padding=1)
        (conv): Conv2dPad(32, 96, kernel_size=(1, 1), stride=(1, 1))
      )
      (3): Sequential(
        (conv): Conv2dPad(32, 96, kernel_size=(1, 1), stride=(1, 1))
      )
    )
  )
)

In [48]:
InceptionBBlock(32)

InceptionBBlock(
  (block): InputForward(
    (blocks): ModuleList(
      (0): Sequential(
        (conv1): Conv2dPad(32, 192, kernel_size=(1, 1), stride=(1, 1))
        (conv2): Conv2dPad(192, 192, kernel_size=(1, 7), stride=(1, 1), padding=(0, 3))
        (conv3): Conv2dPad(192, 224, kernel_size=(7, 1), stride=(1, 1), padding=(3, 0))
        (conv4): Conv2dPad(224, 224, kernel_size=(1, 7), stride=(1, 1), padding=(0, 3))
        (conv5): Conv2dPad(224, 256, kernel_size=(7, 1), stride=(1, 1), padding=(3, 0))
      )
      (1): Sequential(
        (conv1): Conv2dPad(32, 64, kernel_size=(1, 1), stride=(1, 1))
        (conv2): Conv2dPad(64, 96, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      )
      (2): Sequential(
        (pool): AvgPool2d(kernel_size=3, stride=1, padding=1)
        (conv): Conv2dPad(32, 96, kernel_size=(1, 1), stride=(1, 1))
      )
      (3): Sequential(
        (conv): Conv2dPad(32, 96, kernel_size=(1, 1), stride=(1, 1))
      )
    )
  )
)

In [50]:
InceptionCBlock(32)

InceptionCBlock(
  (block): InputForward(
    (blocks): ModuleList(
      (0): Sequential(
        (poll): AvgPool2d(kernel_size=3, stride=1, padding=1)
        (conv1): Conv2dPad(32, 256, kernel_size=(1, 1), stride=(1, 1))
      )
      (1): Sequential(
        (conv1): Conv2dPad(32, 256, kernel_size=(1, 1), stride=(1, 1))
      )
    )
  )
)